Team member 1 (name, email, id): Mhd Jawad Al Rahwanji, mhal00002@stud.uni-saarland.de, 7038980

Team member 2 (name, email, id): Christian Singer, chsi00002@stud.uni-saarland.de, 7039059

## 4.4.a Building your own feed-forward network

Import numpy, which is really all we need to create our own NN.

In [1]:
import numpy as np

Recall that our simple neural network consisted of two layers. We also added a `ReLU` function as a non-linearity to the output of our intermediate layer. Given an input $\mathbf{x} \in \mathbb{R}^n $ we have

$\mathbf{h} = f^{(1)}(\mathbf{x}; \mathbf{W},c) = \text{ ReLU }(\mathbf{W}^\mathsf{T} \mathbf{x} + c)$

$\mathbf{y} = f^{(2)}(\mathbf{h}; \mathbf{w},b) = \text{ softmax }( \mathbf{w}^\mathsf{T} \mathbf{h} + b)$

In this exercise you will create your own network. However, we will do it in a way that allows you to specify the depth of network, i.e. we extend our network such that there isn't just one $\mathbf{h}$ intermediate layers, but rather $n$ of them $\mathbf{h}_{i}$ with $i \in \{1,..., n\}$

**NOTE**: You are not allowed to use any built-in functions to calculate the ReLU, Softmax or the forward pass directly.

**NOTE 2**: Remember to include the non-linearity at every layer. Remember to also add the bias to every layer. Finally, remember to apply the softmax in the output layer.

In [2]:
def relu(x):
    """
    Implement the ReLU function as defined in the lecture
    Input: an array of numbers
    Output: ReLU(x)
    """
    return np.maximum(0,x)

In [3]:
def softmax(x):
    """
    Implement the `softmax` function as defined in the lecture
    """
    # Make softmax numerically stable.
    z = x - np.max(x)
    return np.exp(z)/np.sum(np.exp(z))

In [4]:
class FFNetwork:
    """
    Class representing the feed-forward neural network
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """
        # First layer
        self.W = [np.random.randn(hidden_dim, input_dim)]
        self.b = [np.random.randn(hidden_dim)]
        # Hidden layers
        for i in range(hidden_size-2):
            self.W.append(np.random.randn(hidden_dim, hidden_dim))
            self.b.append(np.random.randn(hidden_dim))
        # Last layer
        self.W.append(np.random.randn(output_dim, hidden_dim))
        self.b.append(np.random.randn(output_dim))

    def forward(self, x):
        """
        Args:
        x: input to the neural network
        
        Output:
        `y`, i.e. the prediction of the network
        """
        for i in range(len(self.W)-1):
            x = relu(np.dot(self.W[i], x) + self.b[i])

        return softmax(np.dot(self.W[i+1], x) + self.b[i+1])

Your implementation needs to be compatible with the following test code:

In [5]:
np.random.seed(0)

# A configuration that reflects the example from the lecture
# i.e. our input is of size 2, our intermediate layers are also of size 2,
# and we will only have 1 hidden layer.
network = FFNetwork(2, 2, 3, 1)
out = network.forward([1.,0.])
print(out)
print(out.shape)
print(out.sum())

[0.9518352  0.01096434 0.03720046]
(3,)
1.0


Disclaimer: Do not expect a correct output at this stage, you are simply building the structure of the network.

However, our setup also allows us to create larger networks:

In [6]:
np.random.seed(0)

network = FFNetwork(2, 3, 2, 4)
out = network.forward([1.,0.])
print(out)
print(out.shape)
print(out.sum())

[5.16295674e-11 1.00000000e+00]
(2,)
1.0


Some sanity checks:

1. You should be seeing the number of units you specified as output units in your output.
1. The numbers in your outputs should be in the range $[0,1]$
1. The numbers should add up to $1$
1. Varying the structure of the network should not break its functionality.

## 4.4.b Implementing a feed-forward network using `torch`

### 4.4.b.1 Creating the network (1 point)

For this we will be using the `nn` module of `torch`, which contains modules representing types of layers. In your case, the specific relevant module would be that of a *fully connected linear layer*.

We will also be using the `nn.functional` module to take advantage of the built in functions for ReLU and Softmax. In this exercise, you are allowed to use them.

In [7]:
import torch
import torch.nn.functional as F

from torch import nn

In [8]:
class TorchFFNetwork(nn.Module):
    """
    A `torch` version of the network implemented for 4.3.b
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """
        super().__init__()

        self.layers = [nn.Linear(input_dim, hidden_dim)]
        # Hidden layers
        for i in range(hidden_size-2):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
        # Last layer
        self.layers.append(nn.Linear(hidden_dim, output_dim))

        self.layers = nn.Sequential(*self.layers)

    def forward(self, x):
        """ forward pass. """
        for i in range(len(self.layers)-1):
            x = F.relu(self.layers[i](x))

        return F.softmax(self.layers[i+1](x), dim=-1)
 


Your implementation, once more, needs to be compatible with the following test code:

In [9]:
torch_network = TorchFFNetwork(2, 3, 2, 1)

In [10]:
out = torch_network(torch.tensor([[1.,0.], [2.,2.]]))
print(out)

tensor([[0.4670, 0.5330],
        [0.4670, 0.5330]], grad_fn=<SoftmaxBackward0>)


Note that the `forward` method is automatically called when you call your network object.

### 4.4.b.2 Training your network (1 point)

Even though we have not covered how training actually works, we will proceed with the training of the a neural network as a blackbox procedure and we will later on learn the internals of the training process (and even implement them ourselves!).

For now, train a neural network (the one you created above) to learn the XOR operation. You are to create a neural network with the appropriate number of input variables, an intermediate hidden layer with 2 units and an output layer with 2 units.

Notes:
- Please read [this introduction to the optimization loop in PyTorch](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#optimization-loop). It should give you a good overview to what PyTorch needs from you to train a neural network.
- You are to train the network until the network learns the operation. Remember to set your random seeds so the results are reproducible.
- There are many optimizers available and Adam is an optimizer that's more complex than SGD. It has not yet been covered in the lecture but its usage in code is equivalent to that of SGD and performs much better.

In [11]:
# Our training X, where each instance includes an x1 and an x2, (where the operation is defined as x1 XOR x2)
training_x = [[0,0], [0,1], [1,0], [1,1]]

# We have only covered softmax in the lecture, so we format the output as follows:
training_y = [[1,0], [0,1], [0,1], [1,0]]

# The Y is formatted such that its first element corresponds to the probability of the XOR resulting in a 0
# and the second element to the probability of the XOR resulting in a 1

################################################################
X = torch.tensor(training_x, dtype=torch.float)
Y = torch.tensor(training_y, dtype=torch.float)

data = (X,Y)
################################################################

In [12]:
model = TorchFFNetwork(2,2,2,1)
model.parameters()

<generator object Module.parameters at 0x000001E5FB7D3AC0>

In [13]:
# Create the model from the previous class and pick a learning rate
torch.manual_seed(42)
model = TorchFFNetwork(2,2,2,1)
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.01, momentum=0.9
)
loss_fn = nn.MSELoss()

In [14]:
def train_loop(data, model, loss_fn, optimizer, num_epochs=100):
    for epoch in range(num_epochs+1):
        out = model(data[0])
        loss = loss_fn(out, data[1])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (not epoch % 5) or epoch < 10:
            print(f"Loss in epoch {epoch}: {loss}")

In [15]:
train_loop(data, model, loss_fn, optimizer)


Loss in epoch 0: 0.2744489908218384
Loss in epoch 1: 0.274308443069458
Loss in epoch 2: 0.2740419805049896
Loss in epoch 3: 0.27366337180137634
Loss in epoch 4: 0.2731854021549225
Loss in epoch 5: 0.27261993288993835
Loss in epoch 6: 0.2719777524471283
Loss in epoch 7: 0.27126896381378174
Loss in epoch 8: 0.2705026865005493
Loss in epoch 9: 0.2699359059333801
Loss in epoch 10: 0.2693568766117096
Loss in epoch 15: 0.26603254675865173
Loss in epoch 20: 0.2625885605812073
Loss in epoch 25: 0.2594967782497406
Loss in epoch 30: 0.25693631172180176
Loss in epoch 35: 0.25622570514678955
Loss in epoch 40: 0.25579890608787537
Loss in epoch 45: 0.2554698884487152
Loss in epoch 50: 0.2551637589931488
Loss in epoch 55: 0.2548638880252838
Loss in epoch 60: 0.2545751631259918
Loss in epoch 65: 0.2543051242828369
Loss in epoch 70: 0.254057377576828
Loss in epoch 75: 0.25383156538009644
Loss in epoch 80: 0.2536250352859497
Loss in epoch 85: 0.2534346878528595
Loss in epoch 90: 0.2532578408718109
Loss 